In [10]:
## Autoencoder : http://deeplearning.net/tutorial/dA.html ##
import numpy as np
import theano
import theano.tensor as T

from logistic import load_data

In [11]:
class DenoisingAutoencoder(object):
    def __init__(self,input,rng,num_v,num_h):
        self.num_h = num_h
        self.num_v = num_v
        # init weight
        wval = np.asarray(rng.uniform(low=-4 * np.sqrt(6. / (num_h + num_v)),
                                      high=4 * np.sqrt(6. / (num_h + num_v)),
                                      size=(num_v,num_h)),dtype=theano.config.floatX)
        self.w = theano.shared(value=wval,name='w',borrow = True)
        # init visible layer bias
        self.bv = theano.shared(value = np.zeros(num_v,dtype=theano.config.floatX),name='bv',borrow=True)
        # init hidden layer bias
        self.bh = theano.shared(value = np.zeros(num_h,dtype=theano.config.floatX),name='bh',borrow=True)
        # setup weight hidden-output layer connections
        #  -> tied weights
        self.w_ = self.w.T
        self.x = input
        self.params = [self.w,self.bv,self.bh]
        
    def encode(self,x_):
        return T.nnet.sigmoid(T.dot(x_,self.w) + self.bh) ### Notice the use of bh here ###
    
    def decode(self,code):
        return T.nnet.sigmoid(T.dot(code,self.w_) + self.bv) ### Notice the use of bv here ###
    
    def loss(self,y,z):
        return - T.sum(self.x * T.log(z) + (1 - self.x) * T.log(1 - z), axis=1)

    def cost(self,y,z):
        return T.mean(self.loss(y,z))
            

In [12]:
from theano.tensor.shared_randomstreams import RandomStreams

# get corrupted input
rng = np.random.RandomState(123)
theano_rng = RandomStreams(rng.randint(2 ** 30))

def corrupt_x(x,theano_rng,corruption_level):
    return theano_rng.binomial(size=x.shape, n=1,
                                        p=1 - corruption_level,
                                        dtype=theano.config.floatX) * x


In [20]:
x = T.matrix('x')

da = DenoisingAutoencoder(rng=rng,input=x,num_v=28*28,num_h=500)

# corrupt x
x_ = corrupt_x(da.x,theano_rng,0.)

In [21]:
# symbolic expressions for y and z
y = da.encode(x_)
z = da.decode(y)

In [22]:
# setup cost, gradients and updates
cost = da.cost(y,z)

# gradients
gparams = T.grad(cost,da.params)

# learning rate
learning_rate = 0.1

# updates
updates = [ (param, param - (learning_rate*gparam) )
              for param,gparam in zip(da.params,gparams)]

In [7]:
# Load MNIST data
datasets = load_data('mnist.pkl.gz')

batch_size = 20

train_set_x, train_set_y = datasets[0]
valid_set_x, valid_set_y = datasets[1]
test_set_x, test_set_y = datasets[2]

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] / batch_size
n_test_batches = test_set_x.get_value(borrow=True).shape[0] / batch_size

... loading data


In [23]:
index = T.lscalar('index')


# compile train function
train = theano.function(inputs=[index],
                        outputs=cost,
                        updates=updates,
                       givens={ x : train_set_x[index*batch_size : (index+1)*batch_size]}
                       )

In [24]:
# actual training
for j in xrange(100):
    cost_iter = 0
    for i in xrange(n_train_batches):
        cost_iter +=  train(i)
    print 'cost per iteration : ',cost_iter/n_train_batches
    
    

cost per iteration :  63.2154804504
cost per iteration :  55.7913007095
cost per iteration :  54.7836948883
cost per iteration :  54.2756162872
cost per iteration :  53.9338045303
cost per iteration :  53.6743318695
cost per iteration :  53.4639914398
cost per iteration :  53.2869263062
cost per iteration :  53.134205513
cost per iteration :  52.9999676834
cost per iteration :  52.880374469
cost per iteration :  52.7728924377
cost per iteration :  52.6755179123
cost per iteration :  52.586786499
cost per iteration :  52.5054787415
cost per iteration :  52.4305104568
cost per iteration :  52.3609948074
cost per iteration :  52.2962339813
cost per iteration :  52.2356936401
cost per iteration :  52.1789335449
cost per iteration :  52.1255581131
cost per iteration :  52.0752050476
cost per iteration :  52.0275402222
cost per iteration :  51.9822658051
cost per iteration :  51.9391478119
cost per iteration :  51.8980120346
cost per iteration :  51.8587114212
cost per iteration :  51.821117

KeyboardInterrupt: 